In [1]:
import numpy as np
import torch
import torch.utils.data as data
import pandas as pd
from src.error_measures import get_accuracy
from src.models import TwoLayerLeaky, ThreeLayerLeaky
from src.train_model import training_loop

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [3]:
# setting seeds
np.random.seed(445)
torch.manual_seed(445)

In [4]:
batch_size = 64
dtype = torch.float

## Data

In [5]:
data06 = pd.read_csv("../data/processed/target06.csv")

In [6]:
all_data = data.TensorDataset(torch.from_numpy((data06.values[:,:-1] - data06.values[:,:-1].min(0)) / data06.values[:,:-1].ptp(0)).float(), torch.from_numpy(data06.values[:,-1]).float())  # with normalization
train_dataset, test_dataset, valid_dataset = torch.utils.data.random_split(all_data, (round(0.7 * len(all_data)), round(0.2 * len(all_data)), round(0.1 * len(all_data))))

In [7]:
train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=256, shuffle=False, drop_last=False)
valid_loader = data.DataLoader(valid_dataset, batch_size=256, shuffle=False, drop_last=False)

## Prepare model

In [8]:
# model = TwoLayerLeaky(num_inputs=8, num_hidden=1200, num_outputs=1, beta=0.95, num_steps=20)
# print(model)

In [9]:
# training_loop(model, train_loader, valid_loader, device, num_epochs=30)

In [10]:
# get_accuracy(model, test_loader, device=device, pct_close=0.2)

Three layer leaky

In [11]:
# model1 = ThreeLayerLeaky(num_inputs=8, num_hidden=250, num_outputs=1, beta=0.95, num_steps=15)
# print(model1)

In [12]:
# training_loop(model1, train_loader, valid_loader, device, num_epochs=80, validation=True)

In [13]:
# get_accuracy(model1, test_loader, device=device, pct_close=0.2)

In [18]:
hidden = [50, 100, 150, 200, 250]
epochs = [20, 30, 50, 80]
steps = [15, 20, 25]

In [19]:
best = 0.
winner = (0, 0, 0)
for h in hidden:
    for e in epochs:
        for s in steps:
            print(f'Hidden: {h}, epochs: {e}, steps: {s}')
            model = ThreeLayerLeaky(num_inputs=8, num_hidden=h, num_outputs=1, beta=0.95, num_steps=s)
            training_loop(model, train_loader, valid_loader, device, num_epochs=e, validation=True)
            acc = get_accuracy(model, test_loader, device=device, pct_close=0.2)
            if acc > best:
                best = acc
                winner = (h, e, s)

Hidden: 50, epochs: 20, steps: 15


  5%|▌         | 1/20 [00:02<00:50,  2.67s/it]

Accuracy: 7.0520231213872835%


 30%|███       | 6/20 [00:12<00:28,  2.05s/it]

Accuracy: 19.537572254335263%


...

Accuracy: 30.404624277456648%


 95%|█████████▌| 76/80 [04:20<00:14,  3.50s/it]

Accuracy: 32.947976878612714%


100%|██████████| 80/80 [04:35<00:00,  3.44s/it]


Accuracy on validation dataset: 32.02312138728324%


In [20]:
winner

(250, 50, 25)